In [7]:
import numpy as np
from MosaicPlot import MosaicPlot as mp
import Interpolations as pol
import time

def main():
	img = np.array([[0.20966011, 0.46244196, 0.85678337, 0.24892138, 0.65882897, 0.17479739],
	[0.65761386, 0.54535605, 0.20861746, 0.61597225, 0.88594544, 0.17675478],
	[0.81582248, 0.56669197, 0.47976072, 0.71329168, 0.08283209, 0.4012043 ],
	[0.64012971, 0.60011478, 0.65507587, 0.03107767, 0.217446,   0.65748758],
	[0.84188694, 0.74767014, 0.235597,   0.83863067, 0.75734715, 0.22550759],
	[0.12377143, 0.25691292, 0.01799602, 0.1075182, 0.04957209, 0.91041073]])

	img = np.array([
		[0,0,0,3,0,0,0,0],
		[0,3,0,0,0,0,0,3],
		[3,0,3,0,0,0,0,0],
		[0,0,0,5,0,0,6,0],
		[10,7,5,7,10,0,10, 0],
		[10,10,5,7,7,5,10, 0],
		[10,0,0,10,10,7,0, 0],
		[5,7,5,3,0,0,0, 0],
		[5,3,0,5,3,0,5,3],
		[3,5,0,0,0,0,3,4],
		[0,0,0,0,3,0,4,3],
	])

	new_img_nn = pol.resize(img=img, fac_x=15.0, fac_y=15.0, method="nn")
	new_img_bilinear = pol.resize(img=img, fac_x=15.0, fac_y=15.0, method="bilinear2")
	start = time.time()
	new_img_bicubic = pol.resize(img=img, fac_x=15.0, fac_y=15.0, method="bicubic")
	end = time.time()
	print(end-start)

	plot1 = mp(mat_data=new_img_nn)
	plot2 = mp(mat_data=new_img_bilinear)
	plot3 = mp(mat_data=new_img_bicubic)
	plot1.show()
	plot2.show()
	plot3.show()

	#pol.bicubic_refs(img, (200,200), f, f)
    
if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'MosaicPlot'

In [6]:
import matplotlib.pyplot as plt
import numpy as np
# import seaborn as sn

# Setting seaborn look for pyplot
#sn.set()

class MosaicPlot:
    def __init__(self, mat_data):
        self.fig, self.axs = plt.subplots(1, 1)
        self.axs.imshow(mat_data)
        self.axs.set_aspect(1)        
        
    def show(self):
        plt.show(self.fig)

In [5]:
import numpy as np
import math
#import numerico as nc
import time

def resize(img, fac_x, fac_y, method):
    if method == "nn":
        result = nearest_neighbor(img, fac_x, fac_y)
    if method == "bilinear":
        result = bilinear_interpol(img, fac_x, fac_y)
    if method == "bilinear2":
        result = bilinear_interpol_2(img, fac_x, fac_y)
    if method == "bicubic":
        result = bicubic_interpol(img, fac_x, fac_y)

    return result

def nearest_neighbor(data, by_x, by_y):
	new_x = int(np.ceil( data.shape[0]*by_x ))
	new_y = int(np.ceil( data.shape[1]*by_y ))
	
	new_img = np.empty((new_x,  new_y))
		
	dec_ratio_x = 1.0/by_x
	dec_ratio_y = 1.0/by_y
	
	for i in range(0, new_img.shape[0]):
		for j in range(0, new_img.shape[1]):
			back_index_x = int(np.floor(i*dec_ratio_x))
			back_index_y = int(np.floor(j*dec_ratio_y))
			new_img[i, j] = data[back_index_x, back_index_y]
	
	return new_img
	
# Obtiene las referencias al primero de los 4 puntos de 'p'
# de la data antes de redimensionar, o retorna None si p
# esta fuera de los límites
def bilinear_refs(old_data, p, factor_x, factor_y):
	i_floor = np.floor(p[0]/factor_x)
	j_floor = np.floor(p[1]/factor_y)
	i_ceil = np.ceil(p[0]/factor_x)
	j_ceil = np.ceil(p[1]/factor_y)
	
	if i_ceil >= old_data.shape[0] or j_ceil >= old_data.shape[1]:
		return math.nan, math.nan
	else:
		Q_00_x = int(i_floor)
		Q_00_y = int(j_floor)
		
		return Q_00_x, Q_00_y
		
def bilinear_interpol(data, by_x, by_y):
	new_x = int(np.ceil( data.shape[0]*by_x ))
	new_y = int(np.ceil( data.shape[1]*by_y ))
	
	new_img = np.empty((new_x,  new_y))
		
	for i in range(0, new_img.shape[0]):
		for j in range(0, new_img.shape[1]):
			coord = (i, j)
			Q = bilinear_refs(data, coord ,by_x, by_y)
			
			x_0 = Q[0]
			y_0 = Q[1]
			x_1 = Q[0]+1
			y_1 = Q[1]+1
			
			if(x_1 >= data.shape[0]):
				x_1 -= 1
				x_0 -= 1
			
			if(y_1 >= data.shape[1]):
				y_1 -= 1
				y_0 -= 1
			
			if not (math.isnan(x_0) or math.isnan(y_0)):
				factor = 1/((x_1 - x_0)*(y_1 - y_0))
				X = np.array([
				[x_1 - i/by_x],
				[i/by_x - x_0]
				]).T
				
				fQ = np.array([
				[ data[x_0, y_0],  data[x_0, y_1] ],
				[ data[x_1, y_0],  data[x_1, y_1] ]
				])
				
				Y = np.array([
				[y_1 - j/by_y],
				[j/by_y - y_0]
				])
				
				new_img[i, j] = factor*np.matmul(np.matmul(X, fQ) , Y)
	return new_img[:int((data.shape[0]-1)*by_x),:int((data.shape[1]-1)*by_y)]
	
def bilinear_interpol_2(data, by_x, by_y):
	new_x = int(np.ceil( data.shape[0]*by_x ))
	new_y = int(np.ceil( data.shape[1]*by_y ))
	
	new_img = np.empty((new_x,  new_y))
		
	start = time.time()	
	for i in range(0, new_img.shape[0]):
		for j in range(0, new_img.shape[1]):
			coord = (i, j)
			Q = bilinear_refs(data, coord ,by_x, by_y)
			
			x_0 = Q[0]
			y_0 = Q[1]
			x_1 = Q[0]+1
			y_1 = Q[1]+1
			
			if(x_1 >= data.shape[0]):
				x_1 -= 1
				x_0 -= 1
			
			if(y_1 >= data.shape[1]):
				y_1 -= 1
				y_0 -= 1
			
			if not (math.isnan(x_0) or math.isnan(y_0)):
				A = np.array([
				[1.0, x_0, y_0, x_0*y_0],
				[1.0, x_0, y_1, x_0*y_1],
				[1.0, x_1, y_0, x_1*y_0],
				[1.0, x_1, y_1, x_1*y_1]
				])

				fQ = np.array([
				[ data[x_0, y_0] ],
				[ data[x_0, y_1] ],
				[ data[x_1, y_0] ],
				[ data[x_1, y_1] ]
				])
			
			result = np.linalg.solve(A, fQ)
			new_img[i,j] = result[0] + result[1]*i/by_x + result[2]*j/by_y + result[3]*i*j/(by_x*by_y)
		end = time.time()
	#print("Bilinear interpolation exec. time: ", end - start)
	return new_img[:int((data.shape[0]-1)*by_x),:int((data.shape[1]-1)*by_y)]

def w_cub(x):
	val = 0

	if (0 <= np.abs(x)) and (np.abs(x) < 1):
		val = np.abs(x)**3 - 2*np.abs(x)**2 + 1.0
	elif (1 <= np.abs(x)) and (np.abs(x) < 2):
		val = -np.abs(x)**3 + 5*np.abs(x)**2 - 8*np.abs(x) + 4.0
	else:
		val = 0
	return val

def p_bicubic_interpol(data, p_ij):
	q = 0
	for j in range(0, 3):
		v = int(np.floor(p_ij[1] - 1 + j))
		p = 0
		for i in range(0, 3):
			u = int(np.floor(p_ij[0]) - 1 + i)
			p = p + data[u, v]*w_cub(p_ij[0] - u)
		q = q + p*w_cub(p_ij[1] - v)
	return q

def bicubic_interpol(data, by_x, by_y):
	new_data = np.empty((int(data.shape[0]*by_x), int(data.shape[1]*by_y)))

	for i in range(0, new_data.shape[0] - int(by_x)):
		for j in range(0, new_data.shape[1] - int(by_y)):
			new_data[i, j] = p_bicubic_interpol(data, (float(i)/by_x, float(j)/by_y))

	return new_data[:int((data.shape[0]-1)*by_x),:int((data.shape[1]-1)*by_y)]